# Market Making Bot

In [1]:
from pumpfun.transactions import PumpFunTransactions
from market_making.main import load_wallet, execute_trading, create_local_liquidation
import asyncio

In [2]:
mint = "ExSqaSHmVcn3EFSRjS3uVsZezkU1FZkJAjamNt2Npump"

wallet_files = [
    "wallets/wallet_pumpfun.json",
    "wallets/wallet_pumpportal.json",
]

wallets = []
for wallet_file in wallet_files:
    wallet = await load_wallet(wallet_file)
    if wallet:
        wallets.append(wallet)
    else:
        print(f"No se pudo cargar wallet: {wallet_file}")

📂 Iniciando sesión de almacenamiento de wallets...
📂 No se encontraron wallets guardadas
✅ Wallet validada correctamente: GMN2f6Ps...
📂 Cerrando sesión de almacenamiento de wallets...
✅ Sesión de almacenamiento de wallets cerrada correctamente
📂 Iniciando sesión de almacenamiento de wallets...
📂 No se encontraron wallets guardadas
✅ Wallet validada correctamente: 62bzYQAb...
📂 Cerrando sesión de almacenamiento de wallets...
✅ Sesión de almacenamiento de wallets cerrada correctamente


In [3]:
if not wallets:
    raise ValueError("No se encontraron wallets válidas")

trading_configs = [
    {
        "wallet": wallets[0],
        "mint": mint,
        "available_capital": "0.0416", # 7.5 USD
        "iterations": 90,
        "delay": 20
    },
    {
        "wallet": wallets[1],
        "mint": mint,
        "available_capital": "0.0416", # 7.5 USD
        "iterations": 90,
        "delay": 20
    }
]

primary_wallet = wallets[0]

async with PumpFunTransactions(api_key=primary_wallet.api_key) as transactions:
    async def execute_trading_with_wallet(config):
        return await execute_trading(
            transactions=transactions,
            wallet=config["wallet"],
            mint=config["mint"],
            available_capital=config["available_capital"],
            iterations=config["iterations"],
            delay=config["delay"]
        )

    async with asyncio.TaskGroup() as tg:
        tasks = [
            tg.create_task(
                execute_trading_with_wallet(config),
                name=f"wallet_{i+1}"
            )
            for i, config in enumerate(trading_configs)
        ]

    print("Todas las wallets completaron trading")

    for i, task in enumerate(tasks):
        try:
            result = task.result()
            wallet_pubkey = trading_configs[i]["wallet"].wallet_public_key[:8]
            print(f" Wallet {wallet_pubkey}...:")
            print(f"    Trades ejecutados: {len(result)}")
            print(f"    Firmas: {result}")
            print("---")
        except Exception as e:
            print(f"    Error en wallet {i+1}: {e}")

🌐 PumpFun API Client inicializado
   WebSocket: ❌ wss://pumpportal.fun/api/data
   HTTP: ✅ https://pumpportal.fun/api
   API Key: ✅ XAhPyqER...
🔌 Iniciando sesión de transacciones PumpFun...
✅ Sesión HTTP inicializada
🛠️ Creando transacción local: buy 0.000462222222222222222 de ExSqaSHmVcn3EFSRjS3uVsZezkU1FZkJAjamNt2Npump
🛠️ Creando transacción local: buy 0.000462222222222222222 de ExSqaSHmVcn3EFSRjS3uVsZezkU1FZkJAjamNt2Npump
📄 Transacción recibida, firmando localmente...
🖋️ Transacción firmada, enviando a https://api.mainnet-beta.solana.com/...
📄 Transacción recibida, firmando localmente...
🖋️ Transacción firmada, enviando a https://api.mainnet-beta.solana.com/...
✅ Transacción enviada. Firma: 2BFhYs4wQRDB6YjQmFJ8qabgjECFxCbjXEo1UCav8tM7MtoaQ5PUsgSVHVQFq8Ha2XhrX2oHKL7FPnKvrRjLKj2j
✅ Transacción enviada. Firma: 4JBbM8NtW4QZWxJLmmDETbP6GJ6neqxEhwvYFEiGvY525YoUnqRXLP7BnCGF35gEbdWfmjEKV8YjBuZDGhZwL5WL
🛠️ Creando transacción local: buy 0.000462222222222222222 de ExSqaSHmVcn3EFSRjS3uVsZezkU

In [ ]:
if not wallets:
    raise ValueError("No se encontraron wallets válidas")

liquidation_percentages = ["100%", "100%"]
primary_wallet = wallets[0]

async with PumpFunTransactions(api_key=primary_wallet.api_key) as transactions:
    async def liquidate_wallet(wallet, percentage):
        return await create_local_liquidation(
            transactions=transactions,
            keypair=wallet.get_keypair(),
            trade_params={
                "mint": mint,
                "amount": percentage,
                "slippage": "15.0",
                "priority_fee": "0.00001"
            }
        )

    async with asyncio.TaskGroup() as tg:
        tasks = [
            tg.create_task(
                liquidate_wallet(wallet, percentage),
                name=f"wallet_{i+1}"
            )
            for i, (wallet, percentage) in enumerate(zip(wallets, liquidation_percentages))
        ]

    print("Liquidación completada para todas las wallets")

    for i, task in enumerate(tasks):
        try:
            result = task.result()
            wallet_pubkey = wallets[i].wallet_public_key[:8]
            percentage = liquidation_percentages[i]
            print(f"Wallet {wallet_pubkey}... ({percentage}): {result}")
        except Exception as e:
            print(f"Error en wallet {i+1}: {e}")